# FALCON 9 VISUAL ANALYTICS WITH FOLIUM

- **TASK 1:** Mark all launch sites on a map
- **TASK 2:** Mark the success/failed launches for each site on the map
- **TASK 3:** Calculate the distances between a launch site to its proximities

In [89]:
import pandas as pd
import numpy as numpy

import folium
from folium.plugins import MarkerCluster
from folium.features import DivIcon

import wget



## Task 1: Mark all launch sites on a map

In [90]:
spacex_csv_file = wget.download('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv')
spacex_df = pd.read_csv(spacex_csv_file)

In [91]:
spacex_df
spacex_df.head()


,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long
0,1,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Failure (parachute),0,28.562302,-80.577356
1,2,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel o...",0.0,LEO (ISS),NASA (COTS) NRO,Failure (parachute),0,28.562302,-80.577356
2,3,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2+,525.0,LEO (ISS),NASA (COTS),No attempt,0,28.562302,-80.577356
3,4,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356
4,5,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356


In [92]:
spacex_df1 = spacex_df[['Launch Site','class','Lat','Long']]     # variables relevantes para la visualización
spacex_df1.head()

,Launch Site,class,Lat,Long
0,CCAFS LC-40,0,28.562302,-80.577356
1,CCAFS LC-40,0,28.562302,-80.577356
2,CCAFS LC-40,0,28.562302,-80.577356
3,CCAFS LC-40,0,28.562302,-80.577356
4,CCAFS LC-40,0,28.562302,-80.577356


In [93]:
ls_df = spacex_df1.groupby('Launch Site', as_index=False).first()   # first() consigue deja el primer termino del indice agrupado
ls_df

,Launch Site,class,Lat,Long
0,CCAFS LC-40,0,28.562302,-80.577356
1,CCAFS SLC-40,1,28.563197,-80.576820
2,KSC LC-39A,1,28.573255,-80.646895
3,VAFB SLC-4E,0,34.632834,-120.610745


In [94]:
# VAMOS A VER EL NASA JOHNSON SPACE CENTER DE HOUSTON,TEXAS

nasajsc= [29.559684888503615, -95.0830971930759]
import folium 
m = folium.Map(location=nasajsc, zoom_star=10)
m

In [95]:
folium.Circle(
    location= nasajsc,
    radius=3000,
    color="black",
    weight=1,
    fill_opacity=0.1,
    opacity=1,
    fill_color="green",
    fill=True,
    popup="NASA Johnson Space Center",
    tooltip="NASA Johnson Space Center"
).add_to(m)


icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 20; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )

folium.map.Marker(
    nasajsc,
    icon=icon,
).add_to(m)

In [96]:
m

_TODO:_  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map

In [97]:
def launchplace(m, LaunchSite, Lat, Long):
    
    folium.Circle(
        location=(Lat, Long),
        radius=5000,
        color="black",
        weight=1,
        fill_opacity=0.1,
        opacity=1,
        fill_color="green",
        fill=True,
        popup=LaunchSite,
        tooltip=LaunchSite
    ).add_to(m)

    icon = DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 20; color:#d35400;"><b>%s</b></div>' % LaunchSite,
    )

    folium.Marker(
        (Lat, Long),
        icon=icon,
    ).add_to(m)

m= folium.Map(location=nasajsc, zoom_start=4.4)

for index, row in ls_df.iterrows():
    launchplace(m, row['Launch Site'], row['Lat'], row['Long'])

In [98]:
m

# Task 2: Mark the success/failed launches for each site on the map

In [99]:
sx_df = spacex_df[['Launch Site','class','Lat','Long','Landing Outcome']]


In [100]:
n = folium.Map(location=(28.56230, -80.577356), zoom_start=10)

marker_cluster = MarkerCluster().add_to(n)

def launchmap(n, LaunchSite, Lat, Long, clas, LandingOutcome):
    
    folium.Circle(
        location=(Lat, Long),
        radius=5000,
        color="black",
        weight=1,
        fill_opacity=0.005,
        opacity=1,
        fill_color="blue",
        fill=True,
        popup=LaunchSite,
        tooltip=LaunchSite
    ).add_to(n)
    
    icon = DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 25; color:#d35400;"><b>%s</b></div>' % LaunchSite,
    )

    if clas == 0:
        color = "red"
    else:
        color = "green"
    
    folium.Marker(
        (Lat, Long),
        popup=LandingOutcome,
        icon=icon,
        color=color,
    ).add_to(n)


# Iterate over each row in the DataFrame and add markers and circles to the map
for index, row in sx_df.iterrows():
    launchmap(n, row['Launch Site'], row['Lat'], row['Long'], row['class'], row['Landing Outcome'])

In [101]:
n

In [102]:
n = folium.Map(location=nasajsc, zoom_start=4)

# Create a MarkerCluster object to hold all the markers
marker_cluster = MarkerCluster().add_to(n)

def launchmap(n, LaunchSite, Lat, Long, clas, LandingOutcome):
    
    folium.Circle(
        location=(Lat, Long),
        radius=5000,
        color="black",
        weight=1,
        fill_opacity=0.005,
        opacity=1,
        fill_color="blue",
        fill=True,
        popup=LaunchSite,
        tooltip=LaunchSite
    ).add_to(n)
    
    if clas == 0:
        color = "red"
    else:
        color = "green"
    
    icon = folium.Icon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 25; color:#d35400;"><b>%s</b></div>' % LaunchSite,
        color=color
    )
    
    folium.Marker(
        (Lat, Long),
        popup=LaunchSite,
        icon=None,
        color= 'black'    
    ).add_to(n) 

    folium.Marker(
        (Lat, Long),
        popup=LandingOutcome,
        icon=icon,
        color=color
    ).add_to(marker_cluster)  # Add markers to the marker_cluster object

# Iterate over each row in the DataFrame and add markers and circles to the map
for index, row in sx_df.iterrows():
    launchmap(n, row['Launch Site'], row['Lat'], row['Long'], row['class'], row['Landing Outcome'])

n

# TASK 3: Calculate the distances between a launch site to its proximities

Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)m

In [103]:
from folium.plugins import MousePosition

formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

n.add_child(mouse_position)
n 

In [104]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

_TODO:_ Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.

In [105]:

launch_site_lat = ls_df['Lat'][0]
launch_site_lon = ls_df['Long'][0]
coastline_lat = 28.56367  
coastline_lon = -80.57163

distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)
distance_coastline

0.5797581813109574

In [106]:
n= folium.Map(location=[28.562302,-80.577356], zoom_start=15)

def launchmap(n, LaunchSite, Lat, Long, clas, LandingOutcome):
    
    folium.Circle(
        location=(Lat, Long),
        radius=5000,
        color="black",
        weight=1,
        fill_opacity=0.005,
        opacity=1,
        fill_color="blue",
        fill=True,
        popup=LaunchSite,
        tooltip=LaunchSite
    ).add_to(n)
    
    if clas == 0:
        color = "red"
    else:
        color = "green"
    
    icon = folium.Icon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 25; color:#d35400;"><b>%s</b></div>' % LaunchSite,
        color=color
    )
    
    folium.Marker(
        (Lat, Long),
        popup=LaunchSite,
        icon=None,
        color= 'black'    
    ).add_to(n) 

    folium.Marker(
        (Lat, Long),
        popup=LandingOutcome,
        icon=icon,
        color=color
    ).add_to(marker_cluster)  # Add markers to the marker_cluster object

# Iterate over each row in the DataFrame and add markers and circles to the map
for index, row in sx_df.iterrows():
    launchmap(n, row['Launch Site'], row['Lat'], row['Long'], row['class'], row['Landing Outcome'])

distance_marker = folium.Marker(
   (28.56367,-80.57163),
   icon=DivIcon(
       icon_size=(20,20),
       icon_anchor=(0,0),
       html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_coastline),
       )
   )
lines=folium.PolyLine(locations=([coastline_lat,coastline_lon],[28.562302,-80.577356]),color="#FF0000", weight=5).add_to(n)

n.add_child(distance_marker)



n